In [15]:
from sklearn.pipeline import Pipeline, make_pipeline
import seaborn as sns 
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, FunctionTransformer, OneHotEncoder
from sklearn.metrics import r2_score



In [2]:
df = sns.load_dataset('penguins')
df = df.dropna(subset=['body_mass_g']) #quitar nulos en la salida 'y' porque es la variable a predecir

X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y = df['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [3]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('model', LinearRegression())
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(pipeline.named_steps)
print(pipeline.named_steps['imputer'])
print(pipeline.named_steps['model'])



{'imputer': SimpleImputer(strategy='median'), 'model': LinearRegression()}
SimpleImputer(strategy='median')
LinearRegression()


In [4]:
#prediccion ejemplo 
X_new = pd.DataFrame([[39.1, np.nan, 181.0]], columns=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm'])
pipeline.predict(X_new)

array([3209.64419227])

In [5]:
#alternativa con make_pipeline

pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    LinearRegression()
)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(pipeline.named_steps)
print(pipeline.named_steps['simpleimputer'])
print(pipeline.named_steps['linearregression'])


{'simpleimputer': SimpleImputer(strategy='median'), 'linearregression': LinearRegression()}
SimpleImputer(strategy='median')
LinearRegression()


## Pipeline con GridSearchCV

In [6]:
pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('transformer', PowerTransformer()),
    ('scaler', MinMaxScaler()),
    ('model', KNeighborsRegressor())
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

params = {
    'imputer__strategy': ['mean', 'median'],
    'transformer__method': ['yeo-johnson', 'box-cox'],
    'scaler__feature_range': [(0, 1), (0, 2)],
    'model__n_neighbors': np.arange(3,20)
}

grid = GridSearchCV(pipeline, params, scoring='r2')
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print('r2_score',r2_score(y_test, y_pred))
print('grid best params', grid.best_params_)




r2_score 0.8151453148627383
grid best params {'imputer__strategy': 'mean', 'model__n_neighbors': np.int64(11), 'scaler__feature_range': (0, 1), 'transformer__method': 'yeo-johnson'}


In [7]:
X_new = pd.DataFrame([[39.1, np.nan, 181.0]], columns=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm'])
grid.predict(X_new)

array([3461.36363636])

In [8]:
pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('transformer', PowerTransformer()), # En este ejemplo lo hacemos opcional
    ('scaler', MinMaxScaler()), # En este ejemplo lo hacemos opcional   
    ('model', KNeighborsRegressor())
])
params = {
    'imputer__strategy': ['mean', 'median'],
    'transformer': [None, PowerTransformer(method='yeo-johnson'), PowerTransformer(method='box-cox')],
    'scaler': [None, MinMaxScaler(feature_range=(0, 1)), MinMaxScaler(feature_range=(0, 2))],
    'model__n_neighbors': np.arange(3, 20)
}
grid = GridSearchCV(pipeline, params, scoring='r2')
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print('r2_score:', r2_score(y_test, y_pred))
print('grid best params:', grid.best_params_)

r2_score: 0.8253040480659294
grid best params: {'imputer__strategy': 'mean', 'model__n_neighbors': np.int64(18), 'scaler': MinMaxScaler(), 'transformer': None}


In [9]:
#Probando varios modelos
pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('transformer', PowerTransformer()), 
    ('scaler', MinMaxScaler()),  
    ('model', 'placeholder') #Se reemplaza por cada modelo en la busqueda
])
params = [
    # KNN 
    {
        'imputer__strategy': ['mean', 'median'],
        'transformer__method': ['yeo-johnson','box-cox'],
        'scaler__feature_range': [(0, 1), (0, 2)],
        'model': [KNeighborsRegressor()],
        'model__n_neighbors': np.arange(3, 20)
    },
    # Decision Tree
    {
        'imputer__strategy': ['mean', 'median'],
        'transformer__method': ['yeo-johnson','box-cox'],
        'scaler__feature_range': [(0, 1), (0, 2)],
        'model': [DecisionTreeRegressor()],
        'model__max_depth': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    }
]

grid = GridSearchCV(pipeline, params, scoring='r2', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print('r2_score:', r2_score(y_test, y_pred))
print('grid best params:', grid.best_params_)
print('grid results:', pd.DataFrame(grid.cv_results_))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
r2_score: 0.8151453148627383
grid best params: {'imputer__strategy': 'mean', 'model': KNeighborsRegressor(), 'model__n_neighbors': np.int64(11), 'scaler__feature_range': (0, 1), 'transformer__method': 'yeo-johnson'}
grid results:      mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.022811      0.002954         0.007723        0.000745   
1         0.037069      0.004373         0.006915        0.000747   
2         0.018040      0.002064         0.005646        0.000370   
3         0.032736      0.006320         0.007536        0.002538   
4         0.019511      0.005022         0.005587        0.000349   
..             ...           ...              ...             ...   
211       0.041556      0.009185         0.009963        0.006352   
212       0.025007      0.005923         0.009317        0.007171   
213       0.050585      0.028736         0.004970        0.000340   
214       0.0248

## FunctionTransformer

Uso de FunctionTransformer para crear funciones personalizadas que usar en el pipeline

In [10]:
def log_transform(X):
    return np.log(X)


pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('log', FunctionTransformer(log_transform)), 
    ('scaler', MinMaxScaler()),  
    ('model', KNeighborsRegressor()) #Se reemplaza por cada modelo en la busqueda

])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print('r2_score:', r2_score(y_test, y_pred))


r2_score: 0.8134498329740103


## ColumnTransformer

Separar y combinar pipelines para hacer distintos tratamientos a deferentes columnas

In [11]:
df = sns.load_dataset('penguins')
df = df.dropna(subset=['body_mass_g']) #quitar nulos en la salida 'y' porque es la variable a predecir

X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y = df['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [12]:
# pipeline numéricas
numerical_cols = X_train.select_dtypes(include=[np.number]).columns
pipeline_numerical = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', MinMaxScaler())
])
# pipeline categóricas
categorical_cols = X_train.select_dtypes(exclude=[np.number]).columns 
pipeline_categorical = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False))
])
# unir pipelines con el column transformer 
pipeline_all = ColumnTransformer ([
    ('numerical', pipeline_numerical, numerical_cols),
    ('categorical', pipeline_categorical, categorical_cols)])
# pipeline final con el modelo
pipeline = make_pipeline(
    pipeline_all,
    KNeighborsRegressor(n_neighbors=7)
)
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  Index([], dtype='object'))])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=7))])

In [13]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(r2_score(y_test, y_pred))

0.8212291991117089


In [14]:
# remainder 'drop' (por defecto)
from sklearn.preprocessing import StandardScaler

pipeline = ColumnTransformer([
        ('numeric', StandardScaler(), ['bill_length_mm', 'bill_depth_mm']),
        ('categorical', OneHotEncoder(), ['species', 'island']),
    ], remainder='drop'
)

# 'flipper_length_mm' y 'sex' han sido eliminadas y no se han procesado
pd.DataFrame(pipeline.fit_transform(X_train, y_train)).head()

ValueError: A given column is not a column of the dataframe

In [ ]:
# remainder 'passthrough'
from sklearn.preprocessing import StandardScaler

pipeline = ColumnTransformer([
        ('numeric', StandardScaler(), ['bill_length_mm', 'bill_depth_mm']),
        ('categorical', OneHotEncoder(), ['species', 'island']),
    ], remainder='passthrough'
)

# 'flipper_length_mm' y 'sex' se mantienen, pero no se han procesado, simplemente se agregan al resultado final
pd.DataFrame(pipeline.fit_transform(X_train, y_train)).head()

In [ ]:
from sklearn.preprocessing import StandardScaler
# remainder con un preprocesador:
X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y = df['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


pipeline_numerical1 = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7)),
    ('scaler', StandardScaler())
])

pipeline_numerical2 = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7)),
    ('scaler', MinMaxScaler())
])

pipeline = ColumnTransformer([
        ('numeric', pipeline_numerical1, ['bill_length_mm', 'bill_depth_mm']),
    ], remainder=pipeline_numerical2
)
pipeline

In [ ]:
# 'bill_length_mm', 'bill_depth_mm' se les aplica StandardScaler, y a 'flipper_length_mm'  se aplica MinMaxScaler
pd.DataFrame(pipeline.fit_transform(X_train, y_train)).head()

## Transformador personalizado

Para crear transformadores preprocesadores personalizados podemos crear una clase Python

In [25]:
# Transformador personalizado para imprimir los datos e inspeccionarlos despues de cada paso de un pipeline
class Debugger(BaseEstimator, TransformerMixin):

    def __init__(self, title, show_shape=True):
        self.title = title
        self.show_shape = show_shape
        
        
    def fit(self, X, y=None):
        # normalmente aqui se aprende o se calculan parametros a partir de los datos de entrada
        print(f'Ejecutando Debugger.fit {self.title}')
        if self.show_shape:
            print(f'Shape de X:  {X.shape}')
            print(f'X sample: {X[:1]}') # mostrar una fila
        return self # devuelve la instancia Debugger para encadenar en el Pipeline
            
        
    def transform(self, X):
        X_copy = X.copy()
        #Aqui hariamos transformaciones sobre X_copy
        print(f'Ejecutando Debugger.transform {self.title}')
        if self.show_shape:
            print(f'Shape de X:  {X_copy.shape}')
            print(f'X sample: {X_copy[:1]}') # mostrar una fila        
        return X_copy 
    
    def fit_transform(self, X, y = None):
        self.fit(X, y)
        return X

In [26]:
X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y = df['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

pipeline = Pipeline([
    ('debug1', Debugger(title='Datos X sin procesar')),
    
    ('imputer', SimpleImputer()),
    ('debug2', Debugger(title='Datos X tras SimpleImputer')),
      
    ('transformer', PowerTransformer()), # En este ejemplo lo hacemos opcional
    ('debug3', Debugger(title='Datos X tras PowerTransformer')),
      
    ('scaler', MinMaxScaler()), # En este ejemplo lo hacemos opcional   
    ('debug4', Debugger(title='Datos X tran MinMaxScaler')),
    
    ('model', KNeighborsRegressor())
])
pipeline.fit(X_train, y_train)

Ejecutando Debugger.fit Datos X sin procesar
Shape de X:  (273, 3)
X sample:      bill_length_mm  bill_depth_mm  flipper_length_mm
115            42.7           18.3              196.0
Ejecutando Debugger.fit Datos X tras SimpleImputer
Shape de X:  (273, 3)
X sample: [[ 42.7  18.3 196. ]]
Ejecutando Debugger.fit Datos X tras PowerTransformer
Shape de X:  (273, 3)
X sample: [[-0.22916448  0.58947392 -0.30358024]]
Ejecutando Debugger.fit Datos X tran MinMaxScaler
Shape de X:  (273, 3)
X sample: [[0.39914212 0.59376449 0.50336414]]


Pipeline(steps=[('debug1', Debugger(title='Datos X sin procesar')),
                ('imputer', SimpleImputer()),
                ('debug2', Debugger(title='Datos X tras SimpleImputer')),
                ('transformer', PowerTransformer()),
                ('debug3', Debugger(title='Datos X tras PowerTransformer')),
                ('scaler', MinMaxScaler()),
                ('debug4', Debugger(title='Datos X tran MinMaxScaler')),
                ('model', KNeighborsRegressor())])

In [27]:
pipeline.predict(X_test)

Ejecutando Debugger.transform Datos X sin procesar
Shape de X:  (69, 3)
X sample:      bill_length_mm  bill_depth_mm  flipper_length_mm
238            46.2           14.5              209.0
Ejecutando Debugger.transform Datos X tras SimpleImputer
Shape de X:  (69, 3)
X sample: [[ 46.2  14.5 209. ]]
Ejecutando Debugger.transform Datos X tras PowerTransformer
Shape de X:  (69, 3)
X sample: [[ 0.40183346 -1.3019806   0.61141932]]
Ejecutando Debugger.transform Datos X tran MinMaxScaler
Shape de X:  (69, 3)
X sample: [[0.5267286  0.15119674 0.71369712]]


array([4770., 4140., 3890., 3960., 3965., 4355., 4255., 3565., 5440.,
       3980., 4145., 4945., 4700., 5110., 3340., 3600., 4110., 3495.,
       3540., 3245., 3565., 3500., 4460., 3300., 4020., 3915., 4110.,
       3610., 3485., 5010., 4025., 3670., 4050., 4480., 4055., 3710.,
       4015., 4945., 4055., 3870., 4110., 3750., 5600., 5270., 3700.,
       3490., 5510., 5490., 3325., 3265., 4800., 3530., 4800., 3690.,
       3350., 3645., 3160., 5550., 5760., 3960., 4540., 5440., 5300.,
       5365., 3400., 3855., 3470., 4185., 4230.])

## Transformador personalizado para outliers

In [28]:
#class OutlierRemover(BaseEstimator, TransformerMixin):
    
    # def __init__(self, factor=1.5):
    #     self.factor = factor #factor para calcular umbrales inferior y superior (metodo turkey)
        
    # def fit(self, X, y=None):
    #     if not isinstance(X, pd.DataFrame):
    #         X = pd.DataFrame(X)    
    #     self.numerical_cols_ = X.select_dtypes(include=[np.number]).columns
    #     Q1 = X[self.numerical_cols_].quantile(0.25)
    #     Q3 = X[self.numerical_cols_].quantile(0.75)
    #     IQR = Q3 - Q1
    #     self.lower_bound_ = Q1 - self.factor * IQR
    #     self.upper_bound_ = Q3 + self.factor * IQR
    #     return self
    
    # def transform(self, X):
    #     X_copy = X.copy()
    #     filtro = ((X_copy[self.numerical_cols_] < self.lower_bound_)| (X_copy[self.numerical_cols_] > self.upper_bound_)).any(axis=1)
    #     return X_copy[filtro]
        
        

In [33]:
# clase que elimina los outliers, se puede crear una variante que simplemente los reemplace por nan
class OutlierRemover(BaseEstimator, TransformerMixin):
    
    def __init__(self, factor=1.5):
        self.factor = factor 
        
    def fit(self, X, y=None):
       
        Q1 = np.percentile(X, 25, axis=0)
        Q3 = np.percentile(X, 75, axis=0)
        IQR = Q3 - Q1
        
        # cálculo de límites
        self.lower_bound_ = Q1 - self.factor * IQR
        self.upper_bound_ = Q3 + self.factor * IQR
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        #filtro = ~((X_copy < self.lower_bound_) | (X_copy > self.upper_bound_)).any(axis=1)
        filtro = np.all((X_copy >= self.lower_bound_) & (X_copy <= self.upper_bound_), axis=1)
        return X_copy[filtro]

In [34]:
remover = OutlierRemover(factor=0.4)
remover.fit_transform(X_train).shape

(187, 3)

In [38]:
df = sns.load_dataset('penguins')
df = df.dropna(subset=['body_mass_g']) 

X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y = df['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

pipeline = make_pipeline(
    SimpleImputer(),
    Debugger(title='X tras SimpleImputer'),
    OutlierRemover(factor=0.4),
    Debugger(title='X tras Outlier'),
    PowerTransformer(),
    Debugger(title='X tras PowerTransformer'),
    MinMaxScaler(),
    Debugger(title='X tran MinMaxScaler'),
    KNeighborsRegressor()
)
pipeline.fit(X_train, y_train)

Ejecutando Debugger.fit X tras SimpleImputer
Shape de X:  (273, 3)
X sample: [[ 42.7  18.3 196. ]]
Ejecutando Debugger.fit X tras Outlier
Shape de X:  (187, 3)
X sample: [[ 42.7  18.3 196. ]]
Ejecutando Debugger.fit X tras PowerTransformer
Shape de X:  (187, 3)
X sample: [[-0.24245763  0.66723679 -0.22030696]]
Ejecutando Debugger.fit X tran MinMaxScaler
Shape de X:  (187, 3)
X sample: [[0.41653698 0.66619004 0.47450903]]


ValueError: Found input variables with inconsistent numbers of samples: [187, 273]

## Transformador personalizado para crear nuevas features

Ejemplo para crear una nueva columna en el dataset titanic

sibsp + parch + 1

In [40]:
df = sns.load_dataset('titanic')
X = df.drop('alive', axis=1)
y = df['alive']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [41]:
class FamilySizeFeature(BaseEstimator, TransformerMixin):
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy['family_size'] = X_copy['sibsp'] +  X_copy['parch']+1
        return X_copy
    
    def fit(self, X, y=None, **fit_params):
        return self

In [42]:
pipeline = make_pipeline(
    FamilySizeFeature(),
    # agregar mas pasos por ejemplo un column transformer con un pipeline para numericas y otro para categoricas
    # agregar modelo 
    
)
pipeline.fit_transform(X_train, y_train)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone,family_size
331,0,1,male,45.5,0,0,28.5000,S,First,man,True,C,Southampton,True,1
733,0,2,male,23.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,True,1
382,0,3,male,32.0,0,0,7.9250,S,Third,man,True,NaN,Southampton,True,1
704,0,3,male,26.0,1,0,7.8542,S,Third,man,True,NaN,Southampton,False,2
813,0,3,female,6.0,4,2,31.2750,S,Third,child,False,NaN,Southampton,False,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1,3,female,21.0,0,0,7.6500,S,Third,woman,False,NaN,Southampton,True,1
270,0,1,male,NaN,0,0,31.0000,S,First,man,True,NaN,Southampton,True,1
860,0,3,male,41.0,2,0,14.1083,S,Third,man,True,NaN,Southampton,False,3
435,1,1,female,14.0,1,2,120.0000,S,First,child,False,B,Southampton,False,4


In [43]:
pipeline.transform(X_test)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone,family_size
709,1,3,male,NaN,1,1,15.2458,C,Third,man,True,NaN,Cherbourg,False,3
439,0,2,male,31.0,0,0,10.5000,S,Second,man,True,NaN,Southampton,True,1
840,0,3,male,20.0,0,0,7.9250,S,Third,man,True,NaN,Southampton,True,1
720,1,2,female,6.0,0,1,33.0000,S,Second,child,False,NaN,Southampton,False,2
39,1,3,female,14.0,1,0,11.2417,C,Third,child,False,NaN,Cherbourg,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,0,3,male,17.0,0,0,7.1250,S,Third,man,True,NaN,Southampton,True,1
773,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,True,1
25,1,3,female,38.0,1,5,31.3875,S,Third,woman,False,NaN,Southampton,False,7
84,1,2,female,17.0,0,0,10.5000,S,Second,woman,False,NaN,Southampton,True,1


In [ ]:
# export con joblib